In [12]:
import requests,time
import os,sys,cv2,random
from lxml import etree
from PIL import Image
from selenium import webdriver
#设置多个user—agent
User_Agent = random.choice([
        "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
        "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
        "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
        "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
        "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
        "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
        "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
        "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
        "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
        "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
        "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
        "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
        "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36',
    ])
def req(url):
    #获取动态连接池
    # 设置多个 user-agent
    # 从http://www.xicidaili.com/爬取代理，并存储到http_list.txt和https_list.txt的文件中
    l=os.listdir(os.getcwd())
    if "http_list.txt" not in l:
        proxy_url='http://www.xicidaili.com/'
        driver=webdriver.Chrome()
        driver.get(proxy_url)
        t=driver.page_source
        driver.quit()
 
        html=etree.HTML(t)
        http_list=[]
        https_list=[]
        for i in (list(range(3,23)) + list(range(25,45))):
            ip=html.xpath('//*[@id="ip_list"]/tbody/tr[{}]/td[2]/text()'.format(i))[0].strip()
            port=html.xpath('//*[@id="ip_list"]/tbody/tr[{}]/td[3]/text()'.format(i))[0].strip()
            protocol=html.xpath('//*[@id="ip_list"]/tbody/tr[{}]/td[6]/text()'.format(i))[0].strip()
            if protocol=="HTTP":
                http_list.append(protocol+r"://"+ip+":"+port)
            if protocol == "HTTPS":
                https_list.append(protocol + r"://" + ip + ":" + port)
        with open("http_list.txt","w") as p:
            p.write(str(http_list))
        with open("https_list.txt", "w") as p:
            p.write(str(https_list))
        print('生成文件成功')
    else:
        print('开始读取文件')
        with open("http_list.txt") as f:
            http_list = eval(f.read())
        with open("https_list.txt") as f:
            https_list = eval(f.read())
        print('读取文件成功')
        print('输出proxies'+http_list[0])
    return http_list,https_list

def get_html(utl,http_list,https_list):
    try:
        proxies={"http":random.choice(http_list),"https":random.choice(https_list)}
        headers={"User-Agent":User_Agent,"cookie":revert_cookie(cookie)}
        r=requests.get(url,proxies=proxies,headers=headers)
        print(r.status_code)
        return r.text
    except:
        print("更换代理中...")
        while True:
            for n in range(10):
                proxies = {"http": random.choice(list(http_list)), "https": random.choice(list(https_list))}
                headers = {"User-Agent": User_Agent,}
                r = requests.get(url, proxies=proxies, headers=headers)
                if r.status_code==200:
                    print(r.status_code)
                    return r.text
                else:
                    continue
            print('无法连接服务器...')

def get_first_url(data):
    '''提取最初页的内容'''
    html=etree.HTML(data)
    try:
        url_list=html.xpath('//a[@class="sj-ys-box"]/@href')
        page_title=html.xpath('//p[@class="subject-ys-p"]/text()')
    #现在获取到标题名称，那么我们要根据标题名称去建立文件夹
        for title in page_title:
            path='F:\觅元素原图\免扣元素专题'+'\\'+title
            read_dir(path)
    except:
        print('提取内容出错')
    return url_list,page_title


def get_change_page(data):
    '''换页'''
    time.sleep(5)
    html=etree.HTML(data)
    try:
        url=html.xpath('//a[@class="nextpage"]/@href')
    except:
        print('换页失败')
    return url[0]

def get_page_url(data):
    '''提取缩略页'''
    html=etree.HTML(data)
    try:
        url_list=html.xpath('//a[@class="img-wrap"]/@href')
    except:
        print('提取列表页出错')
    return url_list

def get_img_url(data,url):
    '''提取高清大图URL'''
    html=etree.HTML(data)
    try:
        url=html.xpath('//img[@class="show-image"]/@src')[0]
        title=html.xpath('//img[@class="show-image"]/@alt')[0]+'.jpg'
    except:
        print('出现异常图片')
        url='http://pic.51yuansu.com//pic3//cover//03//35//71//5b8dc0a6366bd_610.jpg'
        title='替换异常图片'+'.jpg'
        time.sleep(10)
    return url,title

def get_img(url,file,path,http_list,https_list):
    '''下载图片'''
    file_name=path+'\\'+file
    try:
        proxies={"http":random.choice(http_list),"https":random.choice(https_list)}
        headers={"User-Agent":User_Agent,"cookie":revert_cookie(cookie)}
        img=requests.get(url,proxies=proxies,headers=headers).content
        with open(file_name,'wb') as save_img:
            save_img.write(img)
    except:
        print('下载过程出错，这张图片跳过')
        
def re_backcolor(path,filename,new_path):
    '''图片去除背景色'''
    img_path=path+'\\'+filename
    print(img_path)
    str_list=filename.split('.')
    new_path=new_path+'\\'+str_list[0]+'.png'
    print(new_path)
    img = Image.open(img_path)
    img = img.convert("RGBA")  # 转换获取信息
    pixdata = img.load()
    for y in range(img.size[1]):
        for x in range(img.size[0]):
            if pixdata[x, y][0] > 220 and pixdata[x, y][1] > 220 and pixdata[x, y][2] > 220 and pixdata[x, y][3] > 220:
                pixdata[x, y] = (255, 255, 255, 0)
    img.save(new_path)
    
def read_dir(path):
    if not os.path.exists(path):
        os.mkdir(path)

def get_photolist(url_list,first_title,http_list,https_list):
    count=1
    secondcount=0
    for j in first_url_list:
        print(j)
        flag=1000
        second_html=get_html(j,http_list,https_list)
        next_page=get_change_page(second_html)
        second_url_list=get_page_url(second_html)
        dir_name='F:\觅元素原图\免扣元素专题'+'\\'+first_title[secondcount]
        secondcount=secondcount+1
        while  not next_page is None:
            for k in second_url_list:
                if count%100==0:
                    time.sleep(5)
                third_html=get_html(k,http_list,https_list)
                third_url,third_title=get_img_url(third_html,k)
                get_img(third_url,third_title,dir_name,http_list,https_list)
                print(count)
                print('正在下载{}'.format(third_title))
                count=count+1
            headers={'User-Agent':'Mozilla/5.0(Windows NT 10.0; WOW64;rv:66.0)Gecko/20100101 Firefox/66.0',
                    'Referer':second_html,}
            second_html=get_html(next_page,http_list,https_list)
            current_page=next_page
            print('当前页'+current_page)
            next_page=get_change_page(second_html)
            print('下一页'+next_page)
            second_url_list=get_page_url(second_html)
            if current_page==next_page:
                break
            
        for k in second_url_list:
                print('最后一页')
                if count%100==0:
                    time.sleep(5)
                third_html=get_html(k,http_list,https_list)
                third_url,third_title=get_img_url(third_html,k)
                get_img(third_url,third_title,dir_name,http_list,https_list)
                print(count)
                print('正在下载{}'.format(third_title))
                count=count+1

http_list,https_list=req('http://www.51yuansu.com/subject/118-1.html')
for i in range(4):
    '''专题页一共四页'''
    i=i+1
    first_url='http://www.51yuansu.com/subject//'+str(i)+'/'
    first_html=get_html(first_url,http_list,https_list)
    first_url_list,first_title=get_first_url(first_html)
    get_photolist(first_url_list,first_title,http_list,https_list)


开始读取文件
读取文件成功
输出proxiesHTTP://182.35.86.121:9999
更换代理中...


NameError: name 'url' is not defined